In [67]:
import pandas_datareader.data as web
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler

In [68]:
def process_stock_data(df,df_linkedin):
    df_stock=df
    #rename the columns name
    df_stock.columns = ['high','low','open','close','volume_stock','adj_close_stock']
    # add volatility (high-low)/adj_close_stock
    df_stock['hiLo_vola_stock'] = (df_stock['high'] - df_stock['low']) / df_stock['adj_close_stock'] * 100.0
    # ad daily percentage change
    df_stock['pct_change_stock'] = (df_stock['close'] - df_stock['open']) / df_stock['open'] * 100.0
    
    #scale the stock change
    temp_stock = df_stock[['pct_change_stock']].values.astype(float)
    scaler = StandardScaler().fit(temp_stock)
    scaled_data = scaler.transform(temp_stock)
    df_stock['pct_change_stock_scaled'] = scaled_data
    
    
    
    df_sector_linkedin=df_linkedin
    df_sector_linkedin.set_index(pd.DatetimeIndex(df_sector_linkedin['as_of_date']),inplace=True)
    df_sector_linkedin.drop(['as_of_date'],axis=1,inplace=True)
    df_sector_linkedin_sum_df=pd.DataFrame(columns=['followers_count','employees_on_platform','company_count'])
    df_sector_linkedin_sum_df['followers_count']=df_sector_linkedin.groupby('as_of_date').sum()['followers_count']
    df_sector_linkedin_sum_df['employees_on_platform']=df_sector_linkedin.groupby('as_of_date').sum()['employees_on_platform']
    df_sector_linkedin_sum_df['company_count']=df_sector_linkedin.groupby('as_of_date').count()['dataset_id']
    df_sector_linkedin_sum_df['followers_count']=np.array(df_sector_linkedin_sum_df['followers_count'])/np.array(df_sector_linkedin_sum_df['company_count'])
    df_sector_linkedin_sum_df['employees_on_platform']=np.array(df_sector_linkedin_sum_df['employees_on_platform'])/np.array(df_sector_linkedin_sum_df['company_count'])
    
    for i in range(0,df_sector_linkedin_sum_df.shape[0]):
        if df_sector_linkedin_sum_df['company_count'][i]<df_sector_linkedin_sum_df['company_count'].mean():
            df_sector_linkedin_sum_df['employees_on_platform'][i]=np.nan
            df_sector_linkedin_sum_df['followers_count'][i]=np.nan

    total_df=pd.concat([df_sector_linkedin_sum_df,df_stock],join='outer',axis=1)
    
    total_df[[ "volume_stock", "adj_close_stock", "hiLo_vola_stock", "pct_change_stock", "pct_change_stock_scaled"]] = \
        total_df[[ "volume_stock", "adj_close_stock", "hiLo_vola_stock", "pct_change_stock", "pct_change_stock_scaled"]] \
        .interpolate(method='linear', limit_direction='forward', axis=0)
    
    total_df[['employees_on_platform','followers_count']]=total_df[['employees_on_platform','followers_count']].interpolate(method='linear', limit_direction='both', axis=0)
    
    total_df.drop(columns=['company_count','high','low','open','close'],inplace=True)
    
    
    return total_df



In [69]:
def generate_stock_linkedin_data(sector_name,ETF_name):
    sector_list={
                1:'real_estate',#XLRE
                2:'utilities',#XLU
                3:'it',#XLK
                4:'financial',#XLF
                5:'healthcare',#XLV
                6:'consumer_staples',#XLP
                7:'consumer_discretionary',#XLY
                8:'basic_material',#XLB
                9:'energy',#XLE
                10:'industrials'}#XLI
    start = dt.datetime(2015, 9, 16)
    end =  dt.datetime(2018, 7, 17)
    df_stock_raw=web.DataReader(ETF_name, 'yahoo', start, end)
    sector_file_name=sector_list[sector_name]+'_sector_df.csv'
    df_sector_linkedin=pd.read_csv(sector_file_name)
    output_df=process_stock_data(df_stock_raw,df_sector_linkedin)
    output_df.dropna(axis=0,inplace=True)
    output_df['date']=output_df.index
    return output_df
    

In [70]:
real_estate_merge_df=generate_stock_linkedin_data(1,'XLRE')
real_estate_merge_df.to_csv('real_estate_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [71]:
utilities_merge_df=generate_stock_linkedin_data(2,'XLU')
utilities_merge_df.to_csv('utilities_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [72]:
it_merge_df=generate_stock_linkedin_data(3,'XLK')
it_merge_df.to_csv('it_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
financial_merge_df=generate_stock_linkedin_data(4,'XLF')
financial_merge_df.to_csv('financial_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
healthcare_merge_df=generate_stock_linkedin_data(5,'XLV')
healthcare_merge_df.to_csv('healthcare_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
consumer_staples_merge_df=generate_stock_linkedin_data(6,'XLP')
consumer_staples_merge_df.to_csv('consumer_staples_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
consumer_discretionary_merge_df=generate_stock_linkedin_data(7,'XLY')
consumer_discretionary_merge_df.to_csv('consumer_discretionary_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
basic_material_merge_df=generate_stock_linkedin_data(8,'XLB')
basic_material_merge_df.to_csv('basic_material_merge_df.csv',index=False)

E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Users\xbxb\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


energy_merge_df=generate_stock_linkedin_data(9,'XLE')
energy_merge_df.to_csv('energy_merge_df.csv',index=False)
industrials_merge_df=generate_stock_linkedin_data(10,'XLI')
industrials_merge_df.to_csv('industrials_merge_df.csv',index=False)

In [51]:
def read_merge_data(filename):
    df=pd.read_csv(filename)
    df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
    df['yearmonth']=pd.to_datetime(df['date'], format='%Y/%m').map(lambda x: str(x.year)+"-" + str(x.month))
    df.drop(['date'],axis=1,inplace=True) 
    return df

In [57]:
tempdf=read_merge_data('industrials_merge_df.csv')
tempdf.head()

,followers_count,employees_on_platform,volume_stock,adj_close_stock,hiLo_vola_stock,pct_change_stock,pct_change_stock_scaled,yearmonth
date,,,,,,,,
2015-09-16,35355.190476,3717.47619,1.932090e+07,48.598972,1.152283,0.670888,0.850474,2015-9
2015-09-17,35355.190476,3717.47619,1.897740e+07,48.376884,1.984417,-0.324122,-0.511142,2015-9
2015-09-18,35355.190476,3717.47619,2.016960e+07,47.317154,1.479381,-0.683457,-1.002872,2015-9
2015-09-19,35355.190476,3717.47619,1.684813e+07,47.394683,1.392847,-0.475192,-0.717872,2015-9
2015-09-20,35355.190476,3717.47619,1.352667e+07,47.472211,1.306313,-0.266926,-0.432872,2015-9


In [19]:
# max value in a 10 day windows
df_stock['high'].shift(1).rolling(10).max();
# resample the dataframe use month
df_stock['high'].resample('M').min();